# RECOMMENDATION SYSTEM - CONTENT BASED

In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

In [35]:
airbnb = pd.read_csv('./Desktop/listings-2.csv', usecols = ['neighbourhood_cleansed', 'name', 'space'])
airbnb.head(10)

,name,space,neighbourhood_cleansed
0,"La Casa Gaienmae C Harajuku, Omotesando is nearby",This apartment is 3 bedroom flat shared with t...,Shibuya Ku
1,Oshiage Holiday Apartment,"We are happy to welcome you to our apartment, ...",Sumida Ku
2,Private apt in central Tokyo #203,::::::::::::::::::::::::::::::::::::::::::::::...,Nerima Ku
3,"Cozy flat #203, local area YET 10 mins to shib...","Cozy and Relaxing, at home feeling in a reside...",Setagaya Ku
4,Private apt in central Tokyo #201,::::::::::::::::::::::::::::::::::::::::::::::...,Nerima Ku
5,RELAX AT HOME:Shimo 5'/Shibuya 10',Nice LITTLE house with atmosphere in Tokyo. Be...,Setagaya Ku
6,"Classy room @Shinjuku, Takadanoba",Walkable from Shinjuku station even if you mis...,Shinjuku Ku
7,4F - Near Shinjuku & Shibuya w/Free WiFi,Welcome and thank you for reading the details ...,Shibuya Ku
8,WOMAN ONLY LICENSED ! Cosy & Cat behnd Shibuya,We offer quite and peaceful private 6 tatami r...,Setagaya Ku
9,Home stay with a Japanese language teacher fam...,★*:.★*:. Must Read 2 .:*★.:*★ *:.Your private...,Shinjuku Ku


In [42]:
def print_description(index):
    example = airbnb[airbnb.index == index][['neighbourhood_cleansed', 'name', 'space']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1])
        print('Space:\n',example[2])  

In [43]:
print_description(1)

Sumida Ku
Nama: Oshiage Holiday Apartment
Space:
 We are happy to welcome you to our apartment, located in the heart of Tokyo downtown. This is an authentic Japanese apartment with Tatami mattress room and sleeping on Japanese Futon, like Ryokan style. Fully equipped and convienient kitchen will give you oportunity to feel like at home. Automatic bath tub. Separate toilet with heating seat and washlet. Direct acces from both Narita and Haneda airports. Easy access to most of Tokyo attractions. 10min walk from Oshiage Station, 7min walk from Tobu Hikifune Station, 8min walk from Heisei Hikifune Station. Free internet access. Air conditioning, 2 semi-double futon bed (for 2 person each), LCD 32 inch TV, full kitchen, microwave, toster, electric pot, refrigerator, coffee maker, iron, hair dryer, washing machine, bathroom with a bath tub and shower, gas grill. Cooking utensils and linens provided. Our apartment is located in the center of Tokyo downtown, where you can still taste real trad

In [45]:
airbnb['content'] = airbnb[['name', 'space']].astype(str).apply(lambda x: ' // '.join(x), axis = 1)

In [46]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(airbnb['content'])

In [47]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [48]:
results = {}
for idx, row in airbnb.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], airbnb['neighbourhood_cleansed'][i]) for i in similar_indices]
    results[row['neighbourhood_cleansed']] = similar_items[1:]

In [52]:
def item(neighbourhood_cleansed):
    name   = airbnb.loc[airbnb['neighbourhood_cleansed'] == neighbourhood_cleansed]['content'].tolist()[0].split(' // ')[0]
    desc   = ' \nSpace: ' + airbnb.loc[airbnb['neighbourhood_cleansed'] == neighbourhood_cleansed]['content'].tolist()[0].split(' // ')[1][0:165] + '...'
    prediction = name  + desc
    return prediction

def recommend(kota, num):
    print('Rekomendasi ' + str(num) + ' listing yang mirip dengan ' + item(kota))
    print('---')
    recs = results[kota][:num]
    for rec in recs:
        print('\nHasil : ' + item(rec[1]))

In [54]:
KOTA = input("Masukkan Area yang ingin anda lihat: ")
JUMLAH = int(input("Masukkan Jumlah Pencarian: \n"))
recommend(kota = KOTA ,num = JUMLAH)

Masukkan Area yang ingin anda lihat:  Shinagawa Ku
Masukkan Jumlah Pencarian: 
 10


Rekomendasi 10 listing yang mirip dengan share house 'Miraie' @JR Osaki 
Space: Meet and have a great share life experience with an international marriage family as well as a Japanese person during your stay in Japan.  What is Miraie? The name o...
---

Hasil : 30sec Sta.nearSUMO AKB Shinjuku AB 
Space: Subway is the most common form of transit in Tokyo as there are many convenient stations.  It is both practical and inexpensive. This is my Airbnb website - thanks f...

Hasil : Pine House 2 Bed Room + 2 Toilets 
Space: #HOUSE All-private 2 story apartment (1st and 2nd floor) with kitchen, shower, 2 toilets, and 2 bedrooms #CHECK-IN / CHECK-OUT -Self check-in, self check-out (but yo...

Hasil : Oshiage Holiday Apartment 
Space: We are happy to welcome you to our apartment, located in the heart of Tokyo downtown. This is an authentic Japanese apartment with Tatami mattress room and sleeping ...

Hasil : Oshiage Holiday Apartment 
Space: We are happy to welcome you to our apartment, locat